<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=8fd4c0ace6b16c50415be2a4c9b9c4d0aae46932b4ad6a8aef7de4fb784056f5
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-06 12:41:37
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.05 L
Current:  1.43 C
-------------------
Today PnL: -1.39 L (-0.96%)
Current PnL: -21.44 L (-14.05%)
CY Booked + Current PnL: -8.79 L (-5.76%)
-------------------
Total profit:  1.94 L
Total loss:  -23.38 L
-------------------
Total Booked + Current PnL: 18.19 L (14.48%)
Total Booked PnL: 39.63 L (31.55%)
Curr Year Booked PnL: 12.65 L (8.84%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.32 C
Est FTT PnL: 88.43 L (61.78%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.65%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RELIANCE,1533.00,-6.53,68.0,X-LC,5.82,233548.0,23182.0,5605.0,1.63,11.02,2.40,13.68,37.0,4.14,1.64,29.36,XY25,NTT,REFINERIES
79,UNIONBANK,163.00,-5.43,66.0,M-LC,10.66,171482.0,30642.0,13856.0,-0.48,21.76,8.08,31.59,66.0,2.21,1.21,54.01,XY24,NTT,BANKS
78,TTKPRESTIG,770.00,101.49,51.0,M-SC,1.74,86801.0,-13976.0,14070.0,-0.93,-13.87,16.21,0.09,245.0,-0.99,0.61,14.04,OX40N,NTT,DURABLES
2,ABBOTINDIA,35195.00,-10.45,43.0,X-MC,2.60,88560.0,-2118.0,17021.0,0.54,-2.34,19.22,16.44,101.0,-0.12,0.62,17.37,X40,ATH,PHARMA
33,ICICIGI,2252.93,-13.61,58.0,X-MC,5.55,148614.0,12481.0,18101.0,-1.55,9.17,12.18,22.47,91.0,0.69,1.05,23.34,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
24,EASEMYTRIP,26.40,-15.39,60.0,M-SC,27.24,81577.0,-95238.0,176035.0,5.42,-53.86,215.79,45.70,197.0,-0.54,0.57,6.50,XY24,NTT,TRAVEL
6,ASIANPAINT,3465.66,-8.88,70.0,X-LC,4.64,229856.0,-21912.0,75117.0,5.04,-8.70,32.68,21.13,27.0,-0.29,1.62,23.64,X40,ATH,PAINTS
67,SIS,528.00,2044.56,50.0,H-SC,2.70,86547.0,-24485.0,48094.0,2.54,-22.05,55.57,21.26,156.0,-0.51,0.61,16.93,OX40N,NTT,MISC
58,RELIANCE,1533.00,-6.53,68.0,X-LC,5.82,233548.0,23182.0,5605.0,1.63,11.02,2.40,13.68,37.0,4.14,1.64,29.36,XY25,NTT,REFINERIES
13,BERGEPAINT,680.00,-17.83,55.0,X-MC,1.27,220705.0,-6634.0,54691.0,1.40,-2.92,24.78,21.14,106.0,-0.12,1.55,24.40,XY24,NTT,PAINTS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SFL,1287.00,32.32,47.0,M-SC,4.35,174627.0,-87591.0,157426.0,-5.63,-33.40,90.15,26.63,229.0,-0.56,1.23,8.57,XY24,NTT,MISC
8,ATULAUTO,844.00,3370.27,35.0,M-SC,3.28,155737.0,-41395.0,131224.0,-4.14,-21.00,84.26,45.57,236.0,-0.32,1.10,10.41,XY24,NTT,AUTO
71,SYMPHONY,1306.00,-44.56,44.0,M-SC,17.06,117959.0,-53182.0,53129.0,-3.98,-31.07,45.04,-0.03,196.0,-1.00,0.83,2.84,OX40N,NTT,DURABLES
84,VOLTAS,1530.00,-6.25,31.0,X-MC,4.50,196785.0,5043.0,32706.0,-3.49,2.63,16.62,19.69,99.0,0.15,1.39,9.61,XY25,NTT,AC
46,KPIGREEN,731.05,8.64,58.0,H-SC,7.39,129440.0,4143.0,54779.0,-3.46,3.31,42.32,47.03,141.0,0.08,0.91,62.07,MH,ATH,POWER


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,101.49,51.0,M-SC,1.74,86801.0,-13976.0,14070.0,-0.93,-13.87,16.21,0.09,245.0,-0.99,0.61,14.04,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.79,46.0,H-SC,2.69,222111.0,-47556.0,83891.0,-0.70,-17.64,37.77,13.47,138.0,-0.57,1.56,12.94,XY24,NTT,PAINTS
17,CERA,9475.0,-21.96,53.0,H-SC,1.76,142278.0,-33625.0,75649.0,-1.72,-19.12,53.17,23.89,149.0,-0.44,1.00,22.88,OX40N,NTT,CERAMICS
67,SIS,528.0,2044.56,50.0,H-SC,2.70,86547.0,-24485.0,48094.0,2.54,-22.05,55.57,21.26,156.0,-0.51,0.61,16.93,OX40N,NTT,MISC
42,JCHAC,2282.0,19900.00,61.0,M-SC,1.51,97754.0,-30051.0,30040.0,-0.11,-23.51,30.73,-0.01,232.0,-1.00,0.69,10.69,OX40N,NTT,AC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UNIONBANK,163.0,-5.43,66.0,M-LC,10.66,171482.0,30642.0,13856.0,-0.48,21.76,8.08,31.59,66.0,2.21,1.21,54.01,XY24,NTT,BANKS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,49.16,57.0,M-MC,9.91,235311.0,10349.0,157305.0,-1.11,4.60,66.85,74.53,192.0,0.07,1.66,38.34,XY24,BTT,STEEL
46,KPIGREEN,731.05,8.64,58.0,H-SC,7.39,129440.0,4143.0,54779.0,-3.46,3.31,42.32,47.03,141.0,0.08,0.91,62.07,MH,ATH,POWER
79,UNIONBANK,163.00,-5.43,66.0,M-LC,10.66,171482.0,30642.0,13856.0,-0.48,21.76,8.08,31.59,66.0,2.21,1.21,54.01,XY24,NTT,BANKS
85,WHIRLPOOL,2270.00,-40.03,54.0,M-SC,5.36,124264.0,9466.0,84574.0,-1.72,8.25,68.06,81.92,223.0,0.11,0.87,47.11,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.52,53.0,H-SC,5.84,129839.0,6503.0,125126.0,-0.75,5.27,96.37,106.72,119.0,0.05,0.91,29.88,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WHIRLPOOL,2270.00,-40.03,54.0,M-SC,5.36,124264.0,9466.0,84574.0,-1.72,8.25,68.06,81.92,223.0,0.11,0.87,47.11,XR,NTT,DURABLES
46,KPIGREEN,731.05,8.64,58.0,H-SC,7.39,129440.0,4143.0,54779.0,-3.46,3.31,42.32,47.03,141.0,0.08,0.91,62.07,MH,ATH,POWER
36,INDIAMART,4810.62,-53.52,53.0,H-SC,5.84,129839.0,6503.0,125126.0,-0.75,5.27,96.37,106.72,119.0,0.05,0.91,29.88,AR,ATH,MISC
86,WIPRO,420.00,-15.61,42.0,M-LC,5.49,148372.0,-2573.0,112021.0,0.58,-1.70,75.50,72.51,53.0,-0.02,1.04,4.22,XR,NTT,IT
31,HINDZINC,730.22,24.68,43.0,M-LC,10.42,202971.0,-2105.0,114679.0,-1.43,-1.03,56.50,54.89,52.0,-0.02,1.43,22.82,X5K,ATH,METALS


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-53.17,21.0,X-LC,28.48,147560.0,-128963.0,239032.0,-2.52,-46.64,161.99,39.81,54.0,-0.54,1.04,4.00,XY24,NTT,AUTO
12,BATAINDIA,2096.00,-41.58,25.0,X-SC,6.95,87250.0,-41420.0,86718.0,-1.40,-32.19,99.39,35.20,219.0,-0.48,0.61,0.14,X40,NTT,FOOTWEAR
44,JSWINFRA,342.00,-23.55,28.0,X-MC,4.69,191936.0,-8411.0,38905.0,-1.85,-4.20,20.27,15.23,178.0,-0.22,1.35,26.70,X40N,NTT,REALTY
84,VOLTAS,1530.00,-6.25,31.0,X-MC,4.50,196785.0,5043.0,32706.0,-3.49,2.63,16.62,19.69,99.0,0.15,1.39,9.61,XY25,NTT,AC
23,DMART,5391.45,-19.11,32.0,X-LC,8.30,206200.0,-4207.0,63365.0,-1.40,-2.00,30.73,28.12,38.0,-0.07,1.45,21.15,X40N,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,PGHH,17907.65,-30.94,37.0,X-MC,0.92,199770.0,-1050.0,68841.0,-0.62,-0.52,34.46,33.76,80.0,-0.02,1.41,4.65,X40,ATH,FMCG
41,ITC,452.00,-38.34,45.0,X-LC,1.13,198925.0,-1213.0,22101.0,-0.51,-0.61,11.11,10.44,4.0,-0.05,1.40,4.92,X40,NTT,FMCG
66,SIEMENS,4671.50,-3.68,44.0,H-LC,1.25,154650.0,-31445.0,78918.0,-0.85,-16.90,51.03,25.51,15.0,-0.40,1.09,13.97,AR,ATH,ELECTRICAL
28,HAVELLS,2069.16,-8.16,37.0,X-MC,3.05,306600.0,-21391.0,127914.0,-1.78,-6.52,41.72,32.48,92.0,-0.17,2.16,4.16,X40,ATH,ELECTRICAL
80,UNITDSPR,1644.00,-6.24,66.0,X-LC,4.11,245633.0,11489.0,37140.0,-1.57,4.91,15.12,20.77,86.0,0.31,1.73,12.16,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-50.41,33.0,X-SC,37.76,49036.0,-15970.0,166899.0,-2.59,-24.57,340.36,232.18,198.0,-0.10,0.35,0.00,XY24,NTT,MISC
12,BATAINDIA,2096.00,-41.58,25.0,X-SC,6.95,87250.0,-41420.0,86718.0,-1.40,-32.19,99.39,35.20,219.0,-0.48,0.61,0.14,X40,NTT,FOOTWEAR
51,PAGEIND,51769.93,-29.48,33.0,X-MC,7.24,79700.0,-3810.0,23838.0,-0.71,-4.56,29.91,23.98,82.0,-0.16,0.56,0.99,X40,ATH,APPARELS
20,COLPAL,3726.84,-6.52,42.0,X-MC,7.33,218280.0,-45085.0,154411.0,-0.37,-17.12,70.74,41.51,84.0,-0.29,1.54,2.43,XY25,ATH,FMCG
3,ACC,3906.00,-38.50,44.0,X-MC,2.44,183180.0,-54771.0,207415.0,-0.04,-23.02,113.23,64.15,174.0,-0.26,1.29,2.75,XY24,BTT,CEMENT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-50.41,33.0,X-SC,37.76,49036.0,-15970.0,166899.0,-2.59,-24.57,340.36,232.18,198.0,-0.10,0.35,0.00,XY24,NTT,MISC
51,PAGEIND,51769.93,-29.48,33.0,X-MC,7.24,79700.0,-3810.0,23838.0,-0.71,-4.56,29.91,23.98,82.0,-0.16,0.56,0.99,X40,ATH,APPARELS
57,RELAXO,1176.00,-40.36,41.0,X-SC,4.31,81282.0,-63878.0,139805.0,-0.20,-44.01,172.00,52.31,136.0,-0.46,0.57,8.80,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-41.58,25.0,X-SC,6.95,87250.0,-41420.0,86718.0,-1.40,-32.19,99.39,35.20,219.0,-0.48,0.61,0.14,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-10.45,43.0,X-MC,2.60,88560.0,-2118.0,17021.0,0.54,-2.34,19.22,16.44,101.0,-0.12,0.62,17.37,X40,ATH,PHARMA


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-29.01,49.0,X-LC,9.87,283542.0,-62414.0,129125.0,0.88,-18.04,45.54,19.28,1.0,-0.48,2.00,4.82,X40,ATH,IT
39,INFY,2275.00,-19.60,50.0,X-LC,5.12,313856.0,968.0,170706.0,0.38,0.31,54.39,54.87,3.0,0.01,2.21,8.62,X40,BTT,IT
41,ITC,452.00,-38.34,45.0,X-LC,1.13,198925.0,-1213.0,22101.0,-0.51,-0.61,11.11,10.44,4.0,-0.05,1.40,4.92,X40,NTT,FMCG
83,VBL,671.64,-16.43,54.0,X-LC,6.87,301136.0,-14706.0,127381.0,0.21,-4.66,42.30,35.67,5.0,-0.12,2.12,8.55,X40N,ATH,FMCG
1,ABB,7934.00,-39.67,53.0,H-LC,5.06,252264.0,-9355.0,128579.0,0.22,-3.58,50.97,45.57,7.0,-0.07,1.78,7.34,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-49.70,36.0,L-SC,27.77,75648.0,-37901.0,77955.0,-2.64,-33.38,103.05,35.27,268.0,-0.49,0.53,85.15,XR,NTT,HOTELS
7,ASIANTILES,137.00,7411.11,66.0,L-SC,10.61,82427.0,-11383.0,87999.0,-1.33,-12.13,106.76,81.67,269.0,-0.13,0.58,59.62,XR,NTT,CERAMICS
49,MASFIN,398.61,-19.60,49.0,H-SC,8.36,91590.0,-6390.0,27990.0,-2.40,-6.52,30.56,22.05,152.0,-0.23,0.64,33.13,XR,ATH,FINANCE
85,WHIRLPOOL,2270.00,-40.03,54.0,M-SC,5.36,124264.0,9466.0,84574.0,-1.72,8.25,68.06,81.92,223.0,0.11,0.87,47.11,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.52,53.0,H-SC,5.84,129839.0,6503.0,125126.0,-0.75,5.27,96.37,106.72,119.0,0.05,0.91,29.88,AR,ATH,MISC


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,REPCOHOME,880.00,-53.06,67.0,H-SC,7.30,270251.0,-16254.0,273575.0,-2.87,-5.67,101.23,89.82,134.0,-0.06,1.90,42.98,XY24,NTT,FINANCE
81,VAIBHAVGBL,521.00,73.55,64.0,H-SC,2.29,150616.0,-32159.0,143221.0,-2.73,-17.60,95.09,60.77,125.0,-0.22,1.06,34.90,XR,NTT,JEWELLERY
62,SAMMAANCAP,326.00,-197.73,71.0,M-SC,33.12,168651.0,19431.0,124751.0,0.63,13.02,73.97,96.62,208.0,0.16,1.19,82.52,XY25,NTT,FINANCE
68,SONACOMS,806.63,-32.06,66.0,M-SC,6.91,84542.0,-16617.0,56618.0,-0.15,-16.43,66.97,39.54,202.0,-0.29,0.60,19.21,AR,ATH,AUTO
46,KPIGREEN,731.05,8.64,58.0,H-SC,7.39,129440.0,4143.0,54779.0,-3.46,3.31,42.32,47.03,141.0,0.08,0.91,62.07,MH,ATH,POWER


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.76
1,25,44.43
2,50,75.53


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.68
LC    33.25
MC    23.03
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.92
X40      16.29
X40N     12.42
XY25     11.96
XR        9.81
AR        7.86
OX40N     6.52
X200      1.79
X5K       1.43
SR        1.05
MH        0.91
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.18
X-LC    22.53
X-MC    18.74
M-SC    12.59
X-SC     5.47
M-LC     5.09
H-LC     4.66
H-MC     2.32
M-MC     1.66
L-SC     1.44
L-LC     0.97
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.34,-3.82,37.39
IT,12.46,-18.82,82.48
FINANCE,12.16,-9.36,58.94
MISC,7.69,-18.85,78.79
ELECTRICAL,5.79,-9.92,48.89
PAINTS,5.59,-16.13,33.44
BANKS,4.01,-18.46,87.38
INSURANCE,3.85,-1.16,38.87
AUTO,2.74,-48.21,110.06


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3320519.0,22
XR,1259163.0,13
AR,1172266.0,9
X40,843536.0,12
X40N,702878.0,9
OX40N,557192.0,9
XY25,474859.0,8
SR,272672.0,2
X5K,114679.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3392092.0,24
M-SC,1256208.0,15
X-MC,1212198.0,14
X-LC,1078893.0,13
X-SC,570666.0,6
M-LC,285661.0,4
H-LC,277557.0,3
H-MC,257173.0,2
L-SC,256065.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1190978.0      6
           AR         840695.0      5
           XR         789622.0      7
M-SC       XY24       724307.0      6
X-MC       XY24       569465.0      4
X-LC       X40        459145.0      5
X-SC       X40N       317049.0      4
X-LC       XY24       314850.0      2
X-MC       X40        297673.0      6
H-SC       SR         272672.0      2
           OX40N      243346.0      3
X-LC       X40N       227886.0      3
H-LC       AR         207497.0      2
X-MC       XY25       187117.0      2
H-MC       XY24       182859.0      1
X-SC       XY24       166899.0      1
L-SC       XR         165954.0      2
X-MC       X40N       157943.0      2
M-MC       XY24       157305.0      1
M-SC       OX40N      149421.0      4
           XR         133655.0      2
           XY25       124751.0      1
           AR         124074.0      2
M-LC       X5K        114679.0      1
           XR         112021.0      1
L-SC       OX40N       90111.0      1
X-SC       X40         86718.0      1
X-LC       XY25        77012.0      3
H-MC       OX40N       74314.0      1
H-LC       X200        70060.0      1
L-MC       XR          57911.0      1
H-SC       MH          54779.0      1
M-LC       XY25        45105.0      1
L-LC       XY25        40874.0      1
M-LC       XY24        13856.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
